In [7]:
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from collections import defaultdict
from scipy.stats.stats import pearsonr
import pandas as pd

In [8]:
df_tennis = pd.read_csv('dataset/tennis_matches.csv', sep=',', index_col=0) 

#index_col=False indica di non usare la prima colonna come id, ma come dati
df_male = pd.read_csv('dataset/male_players.csv', sep=',', index_col=False)
df_female = pd.read_csv('dataset/female_players.csv', sep=',', index_col=False) 

# Male Table

Surname cleaning, in the surname are present substring with latters and numbers that clearly rappresents errors. The following code clears the surnames form this strings.

In [9]:
#ci sono cifre nei nomi/cognomi?
import re
re_name = "^^[a-zA-Z]+(([',. -][a-zA-Z ])?[a-zA-Z]*)*$"
wrong_surnames = []
wrong_surnames_indexes = []
for i,row in enumerate(df_male['surname']):
    if not(re.search(re_name, str(row))):
        wrong_surnames.append(str(row))
        wrong_surnames_indexes.append(i)

In [10]:
wrong_surnames

['Garcia G003',
 'Gonzalez G162',
 'Casey C100',
 'Phillips P239',
 'Jonsson J092',
 'Gutierrez G230',
 'Lindholm L174',
 'Turturici T225',
 'Brown B395',
 'Robinson R261',
 'Strengberger S597',
 'Parun P991',
 'Gonzalez G419',
 'Phillips Pd13',
 'Deleuran Skjold Db37',
 'Gomez Gb42',
 'Kocyla Kd20',
 'Weise W055',
 'Ukon U004',
 'Perez P114',
 'Pueski P199',
 'Pueski P223',
 'Robinson R263',
 'Wright W096',
 'Parun P071',
 'Khalfan K735',
 'Moore2',
 'Smith 2',
 'Day 3D',
 'Symington 3D',
 'Post 3Rd',
 'Leroy 2',
 'Bergevi Bl61',
 'Grant Gd13',
 'Hernandez Hb01',
 'Perez P727',
 'Lloyd L770',
 'Kovacs Kb22',
 'Fernandes F729',
 'Lindstrom2',
 'Ul Haq H325',
 'Ukon U033',
 'Ul Haq U022',
 'Goransson G423',
 'Antelo A325',
 'Roman R410',
 'Machkamov M741',
 'Williams W303',
 'Robert Rb04',
 'Fernandez F929',
 'Hassan Hb88',
 'Reyes Rc60',
 'Colorado ',
 'Bye1',
 'Cabrera (Chi)',
 'Borras ',
 'Huttrup ',
 'Mullen ',
 'Kepler ',
 'Martin (Bra)',
 '1',
 'Quintero Aguilar ',
 'Hatzis ',
 'K

In [12]:
#TO DO: Gestire stringa vuota, caso dell'entry con l'1 come cognome (eliminare l'entry?)
correct_surname = []
for wrong_surname in wrong_surnames_indexes:
    words = str(df_male['surname'][wrong_surname]).split(" ")
    #se è =1 togliamo solo il numero dalla parola.
    if len(words)==1:
        for caracter in words[0]:
            if caracter.isdigit():
                words[0] = words[0].replace(str(caracter),"")
        correct_surname.append(words[0])
    #se >1 togliamo la seconda parola se ci sono numeri
    if len(words)>1:
        for word in words:
            if not(re.search(re_name, str(word))):
                words.remove(word)
            #check se tutte le parole hanno len>1: se no toglile
#            if len(word)==1:
 #               words.remove(word)
        correct_surname.append(' '.join(words))

In [13]:
correct_surname

['Garcia',
 'Gonzalez',
 'Casey',
 'Phillips',
 'Jonsson',
 'Gutierrez',
 'Lindholm',
 'Turturici',
 'Brown',
 'Robinson',
 'Strengberger',
 'Parun',
 'Gonzalez',
 'Phillips',
 'Deleuran Skjold',
 'Gomez',
 'Kocyla',
 'Weise',
 'Ukon',
 'Perez',
 'Pueski',
 'Pueski',
 'Robinson',
 'Wright',
 'Parun',
 'Khalfan',
 'Moore',
 'Smith',
 'Day',
 'Symington',
 'Post',
 'Leroy',
 'Bergevi',
 'Grant',
 'Hernandez',
 'Perez',
 'Lloyd',
 'Kovacs',
 'Fernandes',
 'Lindstrom',
 'Ul Haq',
 'Ukon',
 'Ul Haq',
 'Goransson',
 'Antelo',
 'Roman',
 'Machkamov',
 'Williams',
 'Robert',
 'Fernandez',
 'Hassan',
 'Reyes',
 'Colorado',
 'Bye',
 'Cabrera',
 'Borras',
 'Huttrup',
 'Mullen',
 'Kepler',
 'Martin',
 '',
 'Quintero Aguilar',
 'Hatzis',
 'Ktiri',
 'Arabaci',
 'Dous Karpenschif',
 'Paris',
 'Compagnucci',
 'Privara',
 'Poljicak',
 'Kovkin',
 'Odani',
 'Westphal',
 'Faucon',
 'Alimli',
 'Oueslati',
 'Eldiyar Uulu',
 'Paliska',
 'Villoslada',
 'Karol',
 'Raynel',
 'Sodan',
 'Merida Aguilar',
 'Sharma